#Import Python Packages

In [1]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/metatlas/metatlas/helpers/' )
import dill2plots as dp
%matplotlib inline

('Metatlas live in ', '/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/metatlas')
you're running on d9d4d6ec916d at 172.17.0.9 


:0: FutureWarning: IPython widgets are experimental and may change in the future.


#Make blank file-info sheet

Get files from database

In [10]:
files = dp.get_metatlas_files(experiment = '%hedlund%',name = '%hilic%',most_recent = True)

Save the files to an empty fileinfo sheet

In [9]:
dp.make_empty_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/hedlund_empty_fileinfo.tab',files)

#Create metatlas groups from filled in file-info sheet

In [18]:
dp = reload(dp)
g = dp.make_groups_from_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/FileInfoSheet_BHedlund_AllFiles_20160413 (1).txt',store=False)

#Create a new Atlas in metatlas from a spreadsheet

In [34]:
dp = reload(dp)
names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/BrianHedlund_Atlas_NEG 1.csv',
                                       'hedlund_hilic_qexactive_neg',
                                       filetype='csv',
                                       sheetname='',
                                       polarity = 'negative',
                                       store=True)

#Select groups of files to operate on

In [35]:
groups = dp.select_groups_for_analysis(name = '%hedlund%neg%',most_recent = True, remove_empty = True, filter_list = [])
# for g in groups:
#     print g.name

0 20160413_Bhedlund_pHILIC_NEG_GBS_Med_inc_Set1 2016-04-14 17:59:30
1 20160413_Bhedlund_pHILIC_NEG_Blank_Set2 2016-04-13 22:34:24
2 20160413_Bhedlund_pHILIC_NEG_PR_End_Set1 2016-04-14 17:59:36
3 20160413_Bhedlund_pHILIC_NEG_JKG1_PR_Set1 2016-04-14 17:59:33
4 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_Ctrl_Set1 2016-04-14 17:59:32
5 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_4OE_Set1 2016-04-14 17:59:37
6 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_inc_Set2 2016-04-13 22:34:26
7 20160413_Bhedlund_pHILIC_NEG_G233_BMR_Set1 2016-04-14 17:59:19
8 20160413_Bhedlund_pHILIC_NEG_BMR_End_Set1 2016-04-14 17:59:39
9 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_32OE_Set2 2016-04-13 22:34:27
10 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_32OE_Set1 2016-04-14 17:59:20
11 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_Ctrl_Set2 2016-04-13 22:34:26
12 20160413_Bhedlund_pHILIC_NEG_GBS_Med_Ctrl_Set1 2016-04-14 17:59:27
13 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_Set1 2016-04-14 17:59:39
14 20160413_Bhedlund_pHILIC_NEG_PR_Beg_Set1 2016-04-14 17:

#Select Atlas to use

In [36]:
dp = reload(dp)
atlas = dp.get_metatlas_atlas(name='%hedlund_hilic_qexactive_neg%',do_print = True)


0 hedlund_hilic_qexactive_neg 2016-04-27 18:58:44


In [37]:
myAtlas = atlas[0]
myAtlas.name

u'hedlund_hilic_qexactive_neg'

#Export Atlas To Spreadsheet

In [38]:
dp = reload(dp)
df = dp.export_atlas_to_spreadsheet(myAtlas,'/global/homes/b/bpb/Downloads/hedlund_hilic_qexactive_neg.csv')
df.head()

,name,inchi,mono_isotopic_molecular_weight,creation_time,formula,last_modified,unique_id,username,label,rt_min,...,common_name,synonyms,pubchem_compound_id,lipidmaps_id,metacyc_id,hmdb_id,img_abc_id,chebi_id,kegg_id,metatlas_inchi
0,adenine,InChI=1S/C5H5N5/c6-4-3-5(9-1-7-3)10-2-8-4/h1-2...,135.054489,1444322234,C5H5N5,1444322292,2070f5dbf1744165966d6f4e7042df4e,bpb,adenine,4.18,...,Adenine///adenine,6-aminopurine///1H-Purin-6-amine///Adeninimine...,190.0///190,nan,ADENINE,HMDB00034,nan,CHEBI:16708,C00147,InChI=1S/C5H5N5/c6-4-3-5(9-1-7-3)10-2-8-4/h1-2...
1,adenosine,InChI=1S/C10H13N5O4/c11-8-5-9(13-2-12-8)15(3-1...,267.096741,1444322234,C10H13N5O4,1444322292,4de2f8142d8141c9abf6ff058566a99e,bpb,adenosine,4.32,...,Adenosine///adenosine,"(2R,3R,4S,5R)-2-(6-amino-9H-purin-9-yl)-5-(hyd...",60961,nan,ADENOSINE,HMDB00050,nan,CHEBI:16335,C00212,InChI=1S/C10H13N5O4/c11-8-5-9(13-2-12-8)15(3-1...
2,L-alanine,"InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5...",89.047676,1444322244,C3H7NO2,1444322293,8bdeefb3322c48cc93cc5534e375b998,bpb,alanine,10.91,...,L-alanine zwitterion///L-Alanine///L-alanine,L-2-Aminopropanoate///alanine///(S)-2-aminopro...,7311724,nan,L-ALPHA-ALANINE,HMDB00161,nan,CHEBI:16977///CHEBI:57972,C00041,"InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5..."
3,L-arginine,InChI=1S/C6H14N4O2/c7-4(5(11)12)2-1-3-10-6(8)9...,174.111679,1444322244,C6H14N4O2,1444322293,94ecdcbad6d74422b2c3ca8e2ea2f47d,bpb,arginine,20.89,...,L-argininate///L-argininium(1+)///D-argininate...,D-argininium(1+)///L-argininium(2+)///D-2-Amin...,1549104///1549073,nan,CPD-220///ARG,nan,57865,CHEBI:32690///CHEBI:32683///CHEBI:32682///CHEB...,C00062///C00792,InChI=1S/C6H14N4O2/c7-4(5(11)12)2-1-3-10-6(8)9...
4,L-asparagine,"InChI=1S/C4H8N2O3/c5-2(4(8)9)1-3(6)7/h2H,1,5H2...",132.053497,1444322244,C4H8N2O3,1444322293,4ce44cc11f8b4a5cafa852abe3ec1cf2,bpb,asparagine,12.49,...,L-Asparagine///L-asparagine///L-asparagine zwi...,"(2S)-2,4-diamino-4-oxobutanoic acid///Aspartic...",6992089,nan,ASN,HMDB00168,nan,CHEBI:17196///CHEBI:58048,C00152,"InChI=1S/C4H8N2O3/c5-2(4(8)9)1-3(6)7/h2H,1,5H2..."


#Get Data and Save it

In [39]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
dp = reload(dp)
output_filename = '/global/homes/b/bpb/data/hedlund/20160427_Bhedlund_pHILIC_NEG_AllGroups.pkl'
data = dp.get_data_for_groups_and_atlas(groups,myAtlas,output_filename,use_set1 = True)

0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH36.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH37.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH38.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH39.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH40.h5
1 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSILVA_BHedlund_chloroflexi_NEG/20151124_pHILIC_NEG_MSMS_BH_GBSInc_1_blank.h5
1 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSILVA_BHedlund_chloroflexi_NEG/20151124_pHILIC_NEG_MSMS_BH_IncCtrl_4_blank.h5
2 21 /global/project/projectdirs/me

#Load a pre-existing datafile

In [40]:
my_file = output_filename #'/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'
project_label = '/global/homes/b/bpb/data/hedlund/20160427_Bhedlund_pHILIC_NEG_AllGroups'
# data = dp.get_data(my_file)

#Make Supplementary Tables

In [41]:
peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')

#Error bar

In [42]:
dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')

#Make Chromatograms

In [43]:
dp = reload(dp)
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 8, scale_y=True , output_loc=project_label+'/scaled_all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=True , output_loc=project_label+'/scaled_all_files_each_compound')
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 8, scale_y=False , output_loc=project_label+'/all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 6, scale_y=False , output_loc=project_label+'/all_files_each_compound')

5
38
nrows =  15
length of ymax is  3072
5
38
nrows =  15
length of ymax is  3072


#Make Identification Figures

In [44]:
dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')

In [45]:
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
from metatlas import metatlas_objects as metob

In [57]:
fr = metob.FragmentationReference()
spectrum = []
for i in range(10):
    mzp = metob.MzIntensityPair()
    mzp.mz = 1.
    mzp.intensity = 10.
    spectrum.append(mzp)
fr.mz_intensities = spectrum

In [58]:
fr

{'creation_time': '2016-04-28T01:31:06',
 'description': u'No description',
 'enabled': True,
 'head_id': u'a61d84cd9eef40d78eab295caf3551c1',
 'last_modified': '2016-04-28T01:31:06',
 'lcms_run': None,
 'mz_intensities': [{'creation_time': '2016-04-28T01:31:06',
 'description': u'No description',
 'head_id': u'adad191c4813436e91ab18d2eacc32d5',
 'intensity': 10.0,
 'last_modified': '2016-04-28T01:31:06',
 'mz': 1.0,
 'name': u'Untitled',
 'prev_uid': u'',
 'unique_id': u'adad191c4813436e91ab18d2eacc32d5',
 'username': u'bpb'},
                    {'creation_time': '2016-04-28T01:31:06',
 'description': u'No description',
 'head_id': u'43cd908c60e4490d814d3dfbc1ef5206',
 'intensity': 10.0,
 'last_modified': '2016-04-28T01:31:06',
 'mz': 1.0,
 'name': u'Untitled',
 'prev_uid': u'',
 'unique_id': u'43cd908c60e4490d814d3dfbc1ef5206',
 'username': u'bpb'},
                    {'creation_time': '2016-04-28T01:31:06',
 'description': u'No description',
 'head_id': u'7b8a03afd230451498e56a3fc